In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
import numpy as np

(x_train, _), (x_test, _) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)


In [2]:
def add_gaussian_noise(images, mean=0.0, stddev=0.5):
    noise = np.random.normal(mean, stddev, images.shape)
    noisy_images = images + noise
    return np.clip(noisy_images, 0., 1.)

def add_salt_and_pepper_noise(images, salt_prob=0.1, pepper_prob=0.1):
    noisy_images = np.copy(images)
    salt = np.random.choice([0, 1], size=images.shape, p=[1 - salt_prob, salt_prob])
    pepper = np.random.choice([0, 1], size=images.shape, p=[1 - pepper_prob, pepper_prob])
    noisy_images[salt == 1] = 1.0
    noisy_images[pepper == 1] = 0.0
    return noisy_images

def add_patchwise_noise(images, num_patches=10, patch_size=(5, 5)):
    noisy_images = np.copy(images)
    for img in noisy_images:
        for _ in range(num_patches):
            x = np.random.randint(0, img.shape[0] - patch_size[0])
            y = np.random.randint(0, img.shape[1] - patch_size[1])
            img[x:x+patch_size[0], y:y+patch_size[1], :] = np.random.choice([0, 1])
    return noisy_images

In [3]:
gaussian_noisy_train = add_gaussian_noise(x_train)
gaussian_noisy_test = add_gaussian_noise(x_test)

In [4]:
salt_and_pepper_noisy_train = add_salt_and_pepper_noise(x_train)
salt_and_pepper_noisy_test = add_salt_and_pepper_noise(x_test)

In [5]:
patchwise_noisy_train = add_patchwise_noise(x_train)
patchwise_noisy_test = add_patchwise_noise(x_test)

In [6]:
from tensorflow.keras import layers, models

def build_encoder():
    encoder = models.Sequential()
    encoder.add(layers.Input(shape=(28, 28, 1)))
    encoder.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
    encoder.add(layers.MaxPooling2D((2, 2), padding='same'))
    encoder.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    encoder.add(layers.MaxPooling2D((2, 2), padding='same'))
    encoder.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    return encoder

def build_decoder():
    decoder = models.Sequential()
    decoder.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    decoder.add(layers.UpSampling2D((2, 2)))
    decoder.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    decoder.add(layers.UpSampling2D((2, 2)))
    decoder.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
    decoder.add(layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same'))
    return decoder

encoder = build_encoder()
decoder = build_decoder()
autoencoder = models.Sequential([encoder, decoder])

autoencoder.compile(optimizer='adam', loss='binary_crossentropy')


In [ ]:
autoencoder.fit(gaussian_noisy_train, x_train, epochs=10, batch_size=128, validation_data=(gaussian_noisy_test, x_test))

Epoch 1/10
228/469 ━━━━━━━━━━━━━━━━━━━━ 3:36 899ms/step - loss: 0.2719

In [ ]:
import matplotlib.pyplot as plt

# Visualize some results
def display_images(noisy, denoised, original, num_images=10):
    plt.figure(figsize=(20, 4))
    for i in range(num_images):
        # Display noisy images
        ax = plt.subplot(3, num_images, i + 1)
        plt.imshow(noisy[i].reshape(28, 28), cmap='gray')
        plt.axis('off')

        # Display denoised images
        ax = plt.subplot(3, num_images, i + num_images + 1)
        plt.imshow(denoised[i].reshape(28, 28), cmap='gray')
        plt.axis('off')

        # Display original images
        ax = plt.subplot(3, num_images, i + 2*num_images + 1)
        plt.imshow(original[i].reshape(28, 28), cmap='gray')
        plt.axis('off')
    plt.show()

# Get denoised images
denoised_images = autoencoder.predict(gaussian_noisy_test)

# Display comparison
display_images(gaussian_noisy_test, denoised_images, x_test)

In [ ]:
autoencoder.fit(salt_and_pepper_noisy_train, x_train, epochs=10, batch_size=128, validation_data=(salt_and_pepper_noisy_test, x_test))

In [ ]:
import matplotlib.pyplot as plt

# Visualize some results
def display_images(noisy, denoised, original, num_images=10):
    plt.figure(figsize=(20, 4))
    for i in range(num_images):
        # Display noisy images
        ax = plt.subplot(3, num_images, i + 1)
        plt.imshow(noisy[i].reshape(28, 28), cmap='gray')
        plt.axis('off')

        # Display denoised images
        ax = plt.subplot(3, num_images, i + num_images + 1)
        plt.imshow(denoised[i].reshape(28, 28), cmap='gray')
        plt.axis('off')

        # Display original images
        ax = plt.subplot(3, num_images, i + 2*num_images + 1)
        plt.imshow(original[i].reshape(28, 28), cmap='gray')
        plt.axis('off')
    plt.show()

# Get denoised images
denoised_images = autoencoder.predict(salt_and_pepper_noisy_test)

# Display comparison
display_images(salt_and_pepper_noisy_test, denoised_images, x_test)

In [ ]:
autoencoder.fit(patchwise_noisy_train, x_train, epochs=10, batch_size=128, validation_data=(patchwise_noisy_test, x_test))

In [ ]:
import matplotlib.pyplot as plt

# Visualize some results
def display_images(noisy, denoised, original, num_images=10):
    plt.figure(figsize=(20, 4))
    for i in range(num_images):
        # Display noisy images
        ax = plt.subplot(3, num_images, i + 1)
        plt.imshow(noisy[i].reshape(28, 28), cmap='gray')
        plt.axis('off')

        # Display denoised images
        ax = plt.subplot(3, num_images, i + num_images + 1)
        plt.imshow(denoised[i].reshape(28, 28), cmap='gray')
        plt.axis('off')

        # Display original images
        ax = plt.subplot(3, num_images, i + 2*num_images + 1)
        plt.imshow(original[i].reshape(28, 28), cmap='gray')
        plt.axis('off')
    plt.show()

# Get denoised images
denoised_images = autoencoder.predict(patchwise_noisy_test)

# Display comparison
display_images(patchwise_noisy_test, denoised_images, x_test)